In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/av-recommendation-systems/sample_submission_J0OjXLi_DDt3uQN.csv
/kaggle/input/av-recommendation-systems/test_HLxMpl7/test.csv
/kaggle/input/av-recommendation-systems/train_mddNHeX/train.csv
/kaggle/input/av-recommendation-systems/train_mddNHeX/challenge_data.csv


In [34]:
#!pip install turicreate

In [35]:
import numpy as np
import pandas as pd
import gc 
pd.set_option('display.max_columns', 50)
import warnings
import turicreate as tc              #importing turicreate for recommdentation
tc.config.set_num_gpus(1)
warnings.filterwarnings("ignore")

In [36]:
import scipy
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import random
import datetime
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
from sklearn.preprocessing import LabelEncoder

In [37]:
train = pd.read_csv('../input/av-recommendation-systems/train_mddNHeX/train.csv')
test = pd.read_csv('../input/av-recommendation-systems/test_HLxMpl7/test.csv')
challenge = pd.read_csv("../input/av-recommendation-systems/train_mddNHeX/challenge_data.csv")

challenge.rename(columns = {'challenge_ID':'challenge'}, inplace = True)  #replacing column name- challenge id to challenge


In [38]:
combine_set=pd.concat([train,test], ignore_index=True)
le =LabelEncoder()
challenge = challenge[["challenge","programming_language",
                       "challenge_series_ID","total_submissions",
                       "category_id","publish_date"]]
challenge["challenge_series_ID"] = le.fit_transform(challenge["challenge_series_ID"].astype(str))
challenge["total_submissions"] = challenge["total_submissions"].fillna(challenge["total_submissions"].mean()).astype(int)
challenge["category_id"] = challenge["category_id"].fillna(challenge["category_id"].mean()).astype(int)
challenge['publish_date'] = pd.DatetimeIndex(challenge['publish_date']).year


mer_train = pd.merge(train,challenge,on=["challenge"],how="left")
mer_test = pd.merge(test,challenge,on=["challenge"],how="left")
mer_df = mer_train.append(mer_test).reset_index(drop=True)
user_df = mer_df.groupby("user_id")["challenge_series_ID"].agg(lambda x: pd.Series.mode(x)[0]).to_frame()
user_df = user_df.reset_index()
user_df = tc.SFrame(user_df)
challenge_df = tc.SFrame(challenge)

In [39]:
#similarity based recommender with cosine similarity
combine_set_tc=tc.SFrame(combine_set)
m=tc.item_similarity_recommender.create(combine_set_tc, user_id='user_id',
                                                            item_id='challenge',
                                                            target='challenge_sequence',
                                                            user_data=user_df,
                                                            item_data = challenge_df,
                                                            similarity_type = "cosine",
                                                            only_top_k=200,
                                                            )

Warning: Ignoring columns user_sequence;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 1301236 observations with 109264 users and 5606 items.

Data prepared in: 0.987927s

...

| 460.539ms                           | 100              | 5606            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.556754s

In [40]:
results = m.recommend(test["user_id"].unique().tolist(),k=3)["challenge"] #top 3 recommenders result
results[:10]

recommendations finished on 1000/39732 queries. users per second: 27460.5

recommendations finished on 2000/39732 queries. users per second: 27579.4

recommendations finished on 3000/39732 queries. users per second: 27652.8

recommendations finished on 4000/39732 queries. users per second: 27695.1

recommendations finished on 5000/39732 queries. users per second: 27719.7

...

recommendations finished on 36000/39732 queries. users per second: 26144.7

recommendations finished on 37000/39732 queries. users per second: 26179.5

recommendations finished on 38000/39732 queries. users per second: 26157.9

recommendations finished on 39000/39732 queries. users per second: 26148.5

dtype: str
Rows: 10
['CI23848', 'CI23691', 'CI25126', 'CI24530', 'CI23848', 'CI25126', 'CI26953', 'CI26959', 'CI26958', 'CI23648']

In [41]:
submission_sample =pd.read_csv('../input/av-recommendation-systems/sample_submission_J0OjXLi_DDt3uQN.csv')

submission_sample["challenge"] = np.array(results).reshape(-1,1)

submission_sample.to_csv("submit_tc.csv",index=False)

In [42]:
submission_sample

,user_sequence,challenge
0,4577_11,CI23848
1,4577_12,CI23691
2,4577_13,CI25126
3,4578_11,CI24530
4,4578_12,CI23848
...,...,...
119191,113834_12,CI24932
119192,113834_13,CI24936
119193,113838_11,CI24958
119194,113838_12,CI23663
